In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

# Skills

import numpy as np
import pandas as pd
import spacy
import io

from nltk.corpus import stopwords


# Similarity Check Starts
from matplotlib import pyplot as plt
import seaborn as sns 
import re

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [2]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr,laparams=laparams)
#      codec=codec,
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk import word_tokenize, pos_tag, ne_chunk

In [4]:
# Extracting Number

def getPhone(text):
    try:
        pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')
        phone = pattern.findall(text)
        phone = [re.sub(r'[,.]', '', el) for el in phone if len(re.sub(r'[()\-.,\s+]', '', el))>6]
        phone = [re.sub(r'\D$', '', el).strip() for el in phone]
        phone = [el for el in phone if len(re.sub(r'\D','',el)) <= 13 and len(re.sub(r'\D','',el))>=11]
        return phone[0]
    except:
        return None

In [5]:
# Extracting Email

def getEmail(text): 
    
    try:
        pattern = re.compile(r'\S*@\S*')
        email = pattern.findall(text)
        return email[0]
    except:
        return None

In [6]:
# Extracting Skills from JD
nlp = spacy.load('en_core_web_sm')
def extract_skills(resume_text):
    nlp = spacy.load("en_core_web_sm")
    stop_words = set(stopwords.words('english'))
    nlp_text = nlp(resume_text)
#     cleaned = [word for word in  if word not in stop_words]
    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("C:/Users/User/Desktop/Skills_Final (1).csv", encoding='latin-1') 
    
    # extract values
    skills = list(data['Skills'])
    skills = [x.strip() for x in skills]
    skillset = []
        # check for bi-grams and tri-grams (example: machine learning)
    # Noun chunks are “base noun phrases” – flat phrases that have a noun as their head
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # check for bi-grams and tri-grams (example: machine learning)
    for token in nlp_text.noun_chunks:
        token = token.text.lower().split("(")[0].strip()
        if token in skills:
            skillset.append(token)
    
    skills_extracted =  [i.capitalize() for i in set([i.lower() for i in skillset])]
    return skills_extracted


In [7]:
# extract degree

def extract_degree(resume_text):
    nlp_text = nlp(resume_text)
    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    # reading the csv file
    data = pd.read_csv("C:/Users/User/Desktop/Degree.csv", encoding='latin-1') 
    
    # extract values
    degrees = list(data['degree'])
    degreeSet = []
        # check for bi-grams and tri-grams (example: machine learning)
    # Noun chunks are “base noun phrases” – flat phrases that have a noun as their head
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in degrees:
            degreeSet.append(token)
    # check for bi-grams and tri-grams (example: machine learning)
    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in degrees:
            degreeSet.append(token)
    
    degrees_extracted =  [i.capitalize() for i in set([i.lower() for i in degreeSet])]
    return degrees_extracted

In [8]:
# Get Experience

def getExperience(sentences):
    has_experience = False
    quals=['working experience','experience']
    for words in sentences:
        
        for i in range(len(words)):
            
            if(words[i][0].lower() in quals):
#                 has_experience = True
                return 1
    return 0

In [9]:
# Get Requirements

def getRequirements(text):
    
#     sentences=nltk.sent_tokenize(text)
#     sentences=[nltk.word_tokenize(el) for el in sentences]
#     sentences=[nltk.pos_tag(el) for el in sentences]
#     try:
#         sen=[]
#         z=0
#         for words in sentences:
#             for i in range(len(words)):
#                 if(words[i][0].lower()=='requirements') or (words[i][0].lower()=='job requirements')or (words[i][0].lower()=='job requirements:')or (words[i][0].lower()=='job responsibilites:') or (words[i][0].lower()=='job responsibilities') or (words[i][0].lower()=='skills' or (words[i][0].lower()=='responsibility')):
#                     index=[z,i]
#                     break
#             z+=1
        
#         exp=[]
#         for i in sentences[index[0]][index[1]+1:]:
#             if i[0].isalpha() and i[1]=='NNP':
#                 exp.append(i[0])
#         print(exp)
#         return exp        
#     except:
#         return None
    
    sentences=nltk.sent_tokenize(text)
    sentences=[nltk.word_tokenize(el) for el in sentences]
    sentences=[nltk.pos_tag(el) for el in sentences]
    sen=[]
    z=0
    i2=[]
    i1=[]
    quals=['requirements','responsibilities','requirement','responsibility','additional requirement','additional requirements']
    for words in sentences:
        for i in range(len(words)):
            if(words[i][0].lower() in quals):
                i1.append(i)
                i2.append(z)
        z+=1

    i2=list(dict.fromkeys(i2))

    qual=[]
    for i in i2:
        if i==i2[0]:
            qual.append((" ".join([sentences[i][k][0] for k in range(i1[0]+1, len(sentences[i]))])))
        else:
            qual.append((" ".join([sentences[i][k][0] for k in range(len(sentences[i]))])))
        
    print(qual)
    return qual

In [10]:
# s2="b.sc"
# s3="b.s.c"
# s4="bachelor"
# s5="m"
# s6="m.s"
# s7="master"
# s8="ms"
# s9="bs"

def getQual(sentences):
    sen=[]
    z=0
    i2=[]
    i1=[]
    quals=['school','graduated','graduation','education','college','university','institute','subject']
    subjects = ['eee','electrical and electronic engineering','computer science and engineering','computer science','computer science & engineering','cse']
    
    for words in sentences:
        for i in range(len(words)):
            if(words[i][0].lower() in quals):
                i1.append(i)
                i2.append(z)
        z+=1

    i2=list(dict.fromkeys(i2))

    qual=[]
    for i in i2:
        if i==i2[0]:
            qual.append((" ".join([sentences[i][k][0] for k in range(i1[0]+1, len(sentences[i]))])))
        else:
            qual.append((" ".join([sentences[i][k][0] for k in range(len(sentences[i]))])))
    stringQual = ""
    for i in range(len(qual)):
        stringQual+=qual[i]
#     print("Real Qual: ",qual)
#     print("String: ",stringQual)
#     print()
#     print()
    listOfQual = stringQual.split("(")
    listOfQual = [x.strip() for x in listOfQual]
#     print("Real Qual: ",qual)
#     print("LIST OF QUAL :" , listOfQual)
    for i in subjects:
        for j in range(len(listOfQual)):
            if i in listOfQual[j].lower():
#                 print(i.upper())
#                 print()
#                 print()
                return i.upper()
        
    return "Not Found"

In [11]:
# POS Tagging in NLTK is a process to mark up the words in text format for a particular part of a speech 
# based on its definition and context. 
def getDetails(text):
    sentences=nltk.sent_tokenize(text)
    sentences=[nltk.word_tokenize(el) for el in sentences]
    sentences=[nltk.pos_tag(el) for el in sentences]
    phone=str(getPhone(text))
    degree = extract_degree(text)
    mail=getEmail(text)
    qual = getQual(sentences)
    exp = getExperience(sentences)
    skills = extract_skills(text)
    skills = ','.join([str(elem) for elem in skills])
#     exp=getExperience(sentences)
#     quals=getQual(sentences)
#     skills=getSkills(sentences)
#     certis=getCertification(sentences)

    for i in range(len(degree)):
        degree[i] = degree[i].lower()

    highest_degree = 'nan'
    for i in range(len(degree)):
        if degree[i][0].lower() == 'm':
            highest_degree = "Masters"
            break
        elif degree[i][0].lower() == 'b':
            highest_degree = "Bachelors"

    dict={
        'Phone_no': phone,
        'Email':mail,
        'Skills':skills,
        'highest_degree':highest_degree,
        'Degree':degree,
        'Experience':exp,
        'Qualification':qual
#         'Experience': exp,
#         'Qualification':quals,
#         'Skills': skills,
#         'Certifications': certis
    }
    return dict

In [12]:
# File Path ---- C:/Users/User/Desktop/ManualParserCV

# The glob module is a useful part of the Python standard library. 
# glob (short for global) is used to return all file paths that match a specific pattern.
# glob patterns specify sets of filenames with wildcard characters”. 
# These patterns are similar to regular expressions but much simpler.
# The glob. glob returns the list of files with their full path 
# (unlike os. listdir()) and is more powerful than os. listdir that does not use wildcards.
details = [];
allNames =[];
import glob
for file in glob.glob("C:/Users/User/Desktop/ManualParserCV/*.pdf"):  
    allNames.append(file.split("\\")[1].split("_")[1].replace('.pdf',''));
    text=convert_pdf_to_txt(file);
    details.append(getDetails(text));

In [13]:
#  Generating External CSV...
for i in range(0,len(details)):
    for j in details[i]["Phone_no"]:
        print(details[i]["Phone_no"])
        print()
        if(j == '0'):
            details[i]["Phone_no"] = '+88'+" "+details[i]["Phone_no"]
            break
        else:
            break
results = []
for i in range(0,len(details)):
    results.append({"Name":allNames[i],"Email":details[i]['Email'],"Phone_no":str(details[i]['Phone_no']),"Skills":details[i]['Skills'],"Experience":details[i]["Experience"],'Obtained Degrees':details[i]['Degree'],'Highest Educational Degree':details[i]["highest_degree"],"Education Background":details[i]['Qualification']})
import csv
filename = 'ParsedCvInfoNew.csv'
with open(filename,'w',newline='') as f:
    w = csv.DictWriter(f,list(results[0].keys()))
    w.writeheader()
    for r in results:
        w.writerow(r)

+880 1616321760

+880 1687918150

01992420723

8801746969454

01735768721

+88 0185-0179564

None

None

+8801626401274

+8801756849807

+8801798435813

+8801782669276

None

+880-1303105281

01403460836

+88 02 55668200

+880 1785700067

01987972042

01799034839

+8801521109475

+880) 1628172326

+8801747702071

01913215235

+8801680728542

+8801885812649

01791485886

01916362616

01762603021

+880 1674 592 911

+8801737 916060

None

01750514197

01625604871

+8801772242616

+8801748304615

6725805748632

+8801741830654

None

01727338441

+88-01779006296

None

+8801815-593582

01685290796

+8801749953224

+8801676029461

+8801787054856

01785443975

+8801670534143

+8801300939434

+8801611832684

01671593944

01827074757

+8801825642114

+880)1620182777

01903334745

+88 01679-843278

+8801980303239

01752633676

01794689278

+8801708929691

+8801912389418

None

01718347937

None

01990 198144

+8801521485583

01765100003

01717654222

+880-1926424697

None

01796205929

+8801677

In [14]:

dataset = pd.read_csv('ParsedCvInfoNew.csv',encoding='cp1252')
dataset.head(10)

,Name,Email,Phone_no,Skills,Experience,Obtained Degrees,Highest Educational Degree,Education Background
0,MD. ASHIK,sabbirahmedashik101@gmail.com,+880 1616321760,"Matlab,Electronics,Arduino,Creative thinking,R...",0,['b.sc'],Bachelors,EEE
1,MD. FARHAN BIN SHAFIQ,farhanbinshafiq@gmail.com,+880 1687918150,"Html5,Json,Github,Bootstrap,Android,Research,T...",1,"['bs', 'hsc', 'ssc']",Bachelors,COMPUTER SCIENCE AND ENGINEERING
2,NIAMUL ISLAM,plabonislam31@gmail.com,+88 01992420723,"Javascript,Github,C,Opengl,Java,Html,E,Css",0,"['b.sc', 'bachelor', 'ms']",Masters,COMPUTER SCIENCE
3,A S M JAHIR HOSSAIN,hossainzahir2009@gmail.com,8801746969454,"Processing,Verilog,Electronics,Ml,Data process...",0,"['bachelor', 'hsc', 'ssc']",Bachelors,EEE
4,ABDUL AZIZ,azizshaon9@gmail.com,+88 01735768721,"Simple,Mysql,Ide,Nltk,Javascript,Github,Androi...",0,['b.sc'],Bachelors,COMPUTER SCIENCE AND ENGINEERING
5,ABDULLAH AL TAJBIR,tajbirnuva@gmail.com,+88 0185-0179564,"B,Github,C,Self,P,D,J,Html,E,Css,T,S,R",0,['b.sc'],Bachelors,CSE
6,ABDUR RAHMAN AL ABIR,NaN,None,NaN,0,[],NaN,Not Found
7,ABDUR RAKIB,@,None,"Python,Nosql,D,Analytics,E,K,T,S,Sql,Pandas,Al...",0,[],NaN,Not Found
8,ABRAR MAHIR,abrarmahir@iut-dhaka.edu,+8801626401274,"Leadership,Research,Content creation,E,Machine...",1,['bachelor'],Bachelors,Not Found
9,ABU BAKKAR SIDDIQUE,u1504110@student.cuet.ac.bd,+8801756849807,"Sql,Machine learning,C++,Android studio,Leader...",1,['bsc'],Bachelors,Not Found


In [15]:

import glob
for file in glob.glob("C:/Users/User/Desktop/Job Description/*.pdf"):
    text=convert_pdf_to_txt(file);
    extract_skills(text)
    getRequirements(text)
   
job_desc_text=convert_pdf_to_txt("C:/Users/User/Desktop/Job Description/Position_Sr. Programmer.pdf");

['will be to design and develop these applications and to coordinate with the rest of the team working on different layers of the system .', 'Job Responsibilities : • Design and implement solutions according to business requirements .', 'Job Requirements : • Strong problem-solving skills • Strong understanding of fundamentals of programming , data structures and algorithms • Hands-on experience with at least 1 programming language and its eco-system e.g. , Javascript ( Node/Browser ) , C # ( ASP.NET/WinForms ) , Java ( Android/Spring ) , etc .']
[': \uf0b7 Maintain records and files of work and revisions .', 'Others Requirement : Technical writing Strong attention to detail - - Written and verbal communication skills - - Able to write in explanatory and procedural styles for multiple audiences - - - Writing skills - Understanding quality - Basic familiarity with the Software Development Excellent prioritization and multitasking skills 1 Year working experience in Quality Concepts .']
[

In [16]:
jd_df = pd.read_csv('C:/Users/User/Desktop/JD.csv',encoding="latin-1")

In [17]:
over_jd = jd_df['Job Description']

In [18]:
def clean_text(raw):
    '''Case specific to be used with pandas apply method'''
    try:
        # remove carriage returns and new lines
        raw = raw.replace('\r', '')
        raw = raw.replace('\n', '')
        
        # brackets appear in all instances
        raw = raw.replace('[', '')
        raw = raw.replace(']', '')
        raw = raw.replace(')', '')
        raw = raw.replace('(', '')
        
        # removing html tags
        clean_html = re.compile('<.*?>')
        clean_text = re.sub(clean_html, ' ', raw)
        
        # removing duplicate whitespace in between words
        # + ---- means one or more times        
        clean_text = re.sub(" +", " ", clean_text) 
        
        # stripping first and last white space 
        clean_text = clean_text.strip()
        
        # commas had multiple spaces before and after in each instance
        clean_text = re.sub(" , ", ", ", clean_text) 
        
        # eliminating the extra comma after a period
        clean_text = clean_text.replace('.,', '.')
        
        # using try and except due to Nan in the column
    except:
        clean_text = np.nan
        
    return clean_text

In [19]:
over_jd = over_jd.apply(clean_text)
over_jd =pd.DataFrame(over_jd)
over_jd.dropna(inplace=True)
over_jd.reset_index(inplace = True, drop = True)

In [20]:
tokenizer = RegexpTokenizer(r"\w+")
over_jd['word_tokenized'] = over_jd["Job Description"].apply(lambda row: tokenizer.tokenize(row))
print(over_jd['word_tokenized'][0])

['Job', 'DescriptionWe', 'are', 'looking', 'for', 'an', 'engineer', 'responsible', 'for', 'building', 'scalable', 'and', 'resilient', 'systems', 'including', 'anything', 'from', 'back', 'end', 'services', 'to', 'their', 'client', 'end', 'counterparts', 'Your', 'primary', 'responsibilities', 'will', 'be', 'to', 'design', 'and', 'develop', 'these', 'applications', 'and', 'to', 'coordinate', 'with', 'the', 'rest', 'of', 'the', 'team', 'working', 'on', 'different', 'layers', 'of', 'the', 'system', 'Therefore', 'a', 'commitment', 'to', 'collaborative', 'problem', 'solving', 'appropriate', 'design', 'and', 'quality', 'product', 'is', 'essential', 'Job', 'Responsibilities', 'Design', 'and', 'implement', 'solutions', 'according', 'to', 'business', 'requirements', 'Collaborate', 'with', 'team', 'leader', 'and', 'other', 'team', 'members', 'Follow', 'agile', 'methodologies', 'for', 'deliveries', 'Write', 'tested', 'maintainable', 'and', 'industry', 'standard', 'code', 'Take', 'ownership', 'of', 

In [21]:
over_jd['sentence_tokenized'] = over_jd["Job Description"].apply(lambda row: sent_tokenize(row))
print("Raw Job Description")
print()
print(over_jd['Job Description'][0])
print()
print("Tokenized Job Description")
print()
print(over_jd['word_tokenized'][0])

Raw Job Description

Job DescriptionWe are looking for an engineer responsible for building scalable and resilient systems, including anything from back-end services to their client-end counterparts. Your primary responsibilities will be to design and develop these applications and to coordinate with the rest of the team working on different layers of the system. Therefore, a commitment to collaborative problem solving, appropriate design, and quality product is essential.Job Responsibilities: Design and implement solutions according to business requirements. Collaborate with team leader and other team members. Follow agile methodologies for deliveries. Write tested, maintainable and industry-standard code. Take ownership of tasks end-to-end. Ensure visibility and observability of deliverables. Learn, experiment and implement new technologies.Job Requirements: Strong understanding of fundamentals of programming, data structures and algorithms Strong problem-solving skills Han

In [22]:
def stopword_removal(text):
    stop_words = set(stopwords.words('english'))
    cleaned = [word for word in text if word not in stop_words]
    return cleaned

In [23]:
over_jd['clean_jd_words'] = over_jd.word_tokenized.apply(stopword_removal)

In [24]:
# Stemming - removes last few words , often provides incorrect result
# Lemming - same as stemming but provides more accurate result
# Example: Caring , after stemming we will get Car , lemming will give us Care
# stemmed and lemma words were both examined to choose which method was best suited
def stemming(text):
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in text]
    return stemmed

def lemming(text):
    lemmatizer = WordNetLemmatizer()
    lemmed = [lemmatizer.lemmatize(word) for word in text]
    return lemmed


In [25]:
over_jd['clean_jd_stemmed'] = over_jd.clean_jd_words.apply(stemming)
over_jd['clean_jd_lemmed'] = over_jd.clean_jd_words.apply(lemming)
print('After Stemming')
print()
print(over_jd['clean_jd_stemmed'][0])
print('After Lemmatization')
print()
print(over_jd['clean_jd_lemmed'][0])

After Stemming

['job', 'descriptionw', 'look', 'engin', 'respons', 'build', 'scalabl', 'resili', 'system', 'includ', 'anyth', 'back', 'end', 'servic', 'client', 'end', 'counterpart', 'your', 'primari', 'respons', 'design', 'develop', 'applic', 'coordin', 'rest', 'team', 'work', 'differ', 'layer', 'system', 'therefor', 'commit', 'collabor', 'problem', 'solv', 'appropri', 'design', 'qualiti', 'product', 'essenti', 'job', 'respons', 'design', 'implement', 'solut', 'accord', 'busi', 'requir', 'collabor', 'team', 'leader', 'team', 'member', 'follow', 'agil', 'methodolog', 'deliveri', 'write', 'test', 'maintain', 'industri', 'standard', 'code', 'take', 'ownership', 'task', 'end', 'end', 'ensur', 'visibl', 'observ', 'deliver', 'learn', 'experi', 'implement', 'new', 'technolog', 'job', 'requir', 'strong', 'understand', 'fundament', 'program', 'data', 'structur', 'algorithm', 'strong', 'problem', 'solv', 'skill', 'hand', 'experi', 'least', '1', 'program', 'languag', 'eco', 'system', 'e', 'g', 

In [26]:

# Python program to convert a list to string
    
# Function to convert  
def listToString(s): 
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s:
        str1 += ele
        str1 +=","
    # return string  
    return str1 

for i in range(len(dataset['Skills'])):
    dataset['Skills'][i] = "".join(str(dataset["Skills"][i]))
    print("".join(dataset["Skills"][i]))
    print()
    print()
        

for i in range(len(over_jd['clean_jd_lemmed'])):  
    over_jd['clean_jd_lemmed'][i] = listToString(over_jd["clean_jd_lemmed"][i])
    
for i in range(len(over_jd['sentence_tokenized'])):  
    over_jd['sentence_tokenized'][i] = listToString(over_jd["sentence_tokenized"][i])
    


over_jd


    


   

Matlab,Electronics,Arduino,Creative thinking,Robotics,E,Computer engineering


Html5,Json,Github,Bootstrap,Android,Research,Testing,Web design,Rest api,Jquery,Basic,Python,Java,E,Js,Django,Ajax


Javascript,Github,C,Opengl,Java,Html,E,Css


Processing,Verilog,Electronics,Ml,Data processing,Simulation,Research,Hdl,Robotics,S,R


Simple,Mysql,Ide,Nltk,Javascript,Github,Android,Bootstrap,Git,Python,Basic,Postgresql,Pycharm,Object-oriented programming,Html,Css,Django,Numpy


B,Github,C,Self,P,D,J,Html,E,Css,T,S,R


nan


Python,Nosql,D,Analytics,E,K,T,S,Sql,Pandas,Algorithms,P,Data structures,J,Numpy,B,C,Data science,R


Leadership,Research,Content creation,E,Machine learning,Editing


Sql,Machine learning,C++,Android studio,Leadership,Firebase,Decision making,Android,C,Self,Python,Basic,Java,Html,K


Software development,Python,Jquery,Mongodb,Html,Javascript,Basic,Source,Ajax,Pl/sql,Sql,C++,Git,Css,Laravel,Vue.js,Bootstrap,Web development,C,Php


Processing,Simulation,Computer vision,Pyth

,Job Description,word_tokenized,sentence_tokenized,clean_jd_words,clean_jd_stemmed,clean_jd_lemmed
0,Job DescriptionWe are looking for an engineer ...,"[Job, DescriptionWe, are, looking, for, an, en...",Job DescriptionWe are looking for an engineer ...,"[Job, DescriptionWe, looking, engineer, respon...","[job, descriptionw, look, engin, respons, buil...","Job,DescriptionWe,looking,engineer,responsible..."
1,KEY RESPONSIBILITIES As a Coordinator in the P...,"[KEY, RESPONSIBILITIES, As, a, Coordinator, in...",KEY RESPONSIBILITIES As a Coordinator in the P...,"[KEY, RESPONSIBILITIES, As, Coordinator, Platf...","[key, respons, as, coordin, platform, allianc,...","KEY,RESPONSIBILITIES,As,Coordinator,Platform,A..."
2,Job DescriptionWe are looking for a Front-End ...,"[Job, DescriptionWe, are, looking, for, a, Fro...",Job DescriptionWe are looking for a Front-End ...,"[Job, DescriptionWe, looking, Front, End, Deve...","[job, descriptionw, look, front, end, develop,...","Job,DescriptionWe,looking,Front,End,Developer,..."
3,Job DescriptionWe are looking for a Full Stack...,"[Job, DescriptionWe, are, looking, for, a, Ful...",Job DescriptionWe are looking for a Full Stack...,"[Job, DescriptionWe, looking, Full, Stack, Web...","[job, descriptionw, look, full, stack, web, de...","Job,DescriptionWe,looking,Full,Stack,Web,Devel..."
4,Analyst  Tech Consulting  SAPThe opportunity...,"[Analyst, Tech, Consulting, SAPThe, opportunit...",Analyst  Tech Consulting  SAPThe opportunity...,"[Analyst, Tech, Consulting, SAPThe, opportunit...","[analyst, tech, consult, sapth, opportunityey,...","Analyst,Tech,Consulting,SAPThe,opportunityEY,l..."
5,*We are looking for a Front-End Web Developer ...,"[We, are, looking, for, a, Front, End, Web, De...",*We are looking for a Front-End Web Developer ...,"[We, looking, Front, End, Web, Developer, join...","[we, look, front, end, web, develop, join, eng...","We,looking,Front,End,Web,Developer,join,engine..."
6,Requiredqualification andexperience?Must have?...,"[Requiredqualification, andexperience, Must, h...",Requiredqualification andexperience?Must have?...,"[Requiredqualification, andexperience, Must, A...","[requiredqualif, andexperi, must, acquir, degr...","Requiredqualification,andexperience,Must,Acqui..."
7,Job DescriptionWe are looking for a Junior Sof...,"[Job, DescriptionWe, are, looking, for, a, Jun...",Job DescriptionWe are looking for a Junior Sof...,"[Job, DescriptionWe, looking, Junior, Software...","[job, descriptionw, look, junior, softwar, eng...","Job,DescriptionWe,looking,Junior,Software,Engi..."
8,Job Responsibility:? Maintain records and file...,"[Job, Responsibility, Maintain, records, and, ...","Job Responsibility:?,Maintain records and file...","[Job, Responsibility, Maintain, records, files...","[job, respons, maintain, record, file, work, r...","Job,Responsibility,Maintain,record,file,work,r..."
9,Job Responsibility:? Writing and organizing co...,"[Job, Responsibility, Writing, and, organizing...","Job Responsibility:?,Writing and organizing co...","[Job, Responsibility, Writing, organizing, cod...","[job, respons, write, organ, code, program, im...","Job,Responsibility,Writing,organizing,code,pro..."


**Content Based Recommendation**

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

tfv_matrix = tfv.fit_transform(dataset['Skills'])

In [28]:
tfv_matrix.shape
# coldStart problem

(297, 872)

In [29]:
def Jaccard_Similarity(doc1, doc2): 
    
    # List the unique words in a document
    words_doc1 = set(doc1)
    words_doc2 = set(doc2)
    
    # Find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)
#     print(len(intersection))

    # Find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)
#     print(len(union))
        
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)

In [30]:
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [31]:
# Reverse mapping of indices and CVs
indices = pd.Series(dataset.index, index=dataset['Skills']).drop_duplicates()

In [32]:
indices

Skills
Matlab,Electronics,Arduino,Creative thinking,Robotics,E,Computer engineering                                                                               0
Html5,Json,Github,Bootstrap,Android,Research,Testing,Web design,Rest api,Jquery,Basic,Python,Java,E,Js,Django,Ajax                                         1
Javascript,Github,C,Opengl,Java,Html,E,Css                                                                                                                 2
Processing,Verilog,Electronics,Ml,Data processing,Simulation,Research,Hdl,Robotics,S,R                                                                     3
Simple,Mysql,Ide,Nltk,Javascript,Github,Android,Bootstrap,Git,Python,Basic,Postgresql,Pycharm,Object-oriented programming,Html,Css,Django,Numpy            4
                                                                                                                                                        ... 
nan                                                

In [33]:
def give_rec(cv, sig=sig):
    idx = indices[cv]
 
    sig_scores = list(enumerate(sig[idx]))

    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    sig_scores = sig_scores[1:11]

    cv_indices = [i[0] for i in sig_scores]

    return dataset['Skills'].iloc[cv_indices]


In [34]:
# Jaccard Similarity 
job_desc = ','.join([str(elem) for elem in extract_skills(job_desc_text)])
print("JOB DESCRIPTION")
print(job_desc)
print()
print()
print()

# set_job_desc = set(job_desc)
# set_cv = set(new_df3['clean_overview'][0])
# print(set_job_desc.intersection(set_cv))
# print(len(set_job_desc) / len(set_cv))
print("Skills")
print(dataset['Skills'][1])
Jaccard_Similarity(dataset['Skills'][1],job_desc)


JOB DESCRIPTION
Sql,Javascript,C,Basic,Js



Skills
Html5,Json,Github,Bootstrap,Android,Research,Testing,Web design,Rest api,Jquery,Basic,Python,Java,E,Js,Django,Ajax


0.34210526315789475

In [35]:
# Cosine Similarity
from sklearn.feature_extraction.text import CountVectorizer 
cv = CountVectorizer() 

from sklearn.metrics.pairwise import cosine_similarity 
#Print the similarity scores 
Scores = []
for i in range(len(dataset["Skills"])):
    text = [dataset["Skills"][i],job_desc]
    count_matrix = cv.fit_transform(text) 
    Scores.append(float(cosine_similarity(count_matrix)[0][1]))
#     Scores2.append({"CV":i,"Accuracy":float(cosine_similarity(count_matrix)[0][1])})

    
    
# Scores_sorted =  sorted(Scores2, key = lambda i: i['Accuracy'],reverse=True)
# print("LOOOPPPP-----",Scores_sorted[5]["Accuracy"])
print(Scores)
max_index = Scores.index(max(Scores))
print(Scores[max_index])
print("=============JD===========",job_desc)
print("=============CV===========",dataset["Skills"][max_index])


text = [dataset["Skills"][max_index],dataset["Skills"][0]]
count_matrix = cv.fit_transform(text) 
give_rec(dataset["Skills"][0])

# Figure out what sigmoid kernel actually does
# Check the similarities between job desc with the recommended CVs
# Top 10 CVs ? Fix a Threshold value
# Try different methods
# Reliability checking with real HR person

[0.0, 0.23570226039551587, 0.20412414523193154, 0.0, 0.22360679774997896, 0.0, 0.0, 0.1386750490563073, 0.0, 0.24253562503633297, 0.4902903378454601, 0.0944911182523068, 0.0, 0.0, 0.24253562503633297, 0.12126781251816648, 0.15811388300841897, 0.12909944487358055, 0.0, 0.3481553119113957, 0.09622504486493763, 0.16666666666666666, 0.1414213562373095, 0.0, 0.15811388300841897, 0.2182178902359924, 0.12126781251816648, 0.0, 0.14744195615489714, 0.14433756729740646, 0.0, 0.3779644730092272, 0.44194173824159216, 0.10425720702853739, 0.17677669529663687, 0.10425720702853739, 0.09622504486493763, 0.3380617018914066, 0.4082482904638631, 0.0, 0.0, 0.45226701686664544, 0.20412414523193154, 0.09284766908852593, 0.1091089451179962, 0.23570226039551587, 0.0, 0.0, 0.16666666666666666, 0.0, 0.1889822365046136, 0.2773500981126146, 0.3333333333333333, 0.0, 0.0, 0.11785113019775793, 0.0, 0.1386750490563073, 0.0, 0.0, 0.0, 0.08838834764831843, 0.1796053020267749, 0.5, 0.20412414523193154, 0.306186217847897

224    Matlab,Electronics,Arduino,Fluent,Testing,Mult...
175    Matlab,Electronics,Arduino,Computer literacy,R...
294    Matlab,Electronics,Arduino,Verilog,Simulation,...
160    Electronics,Arduino,Automation,C,Erp,Prototype...
46     Electronics,Arduino,Red,Raspberry pi,Research,...
137    Matlab,Electronics,Arduino,Simulation,Enclosur...
155    Matlab,Electronics,Arduino,Electronic engineer...
88     Matlab,Arduino,C++,Pilot,C,Simulink,E,Distribu...
209    Electronic engineering,Matlab,Microsoft office...
277    Matlab,Electronics,Arduino,Harbour,Coordinatin...
Name: Skills, dtype: object

In [36]:
# Semantic Similarity
!pip install --upgrade gensim

import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

# Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")    
similarity_index = WordEmbeddingSimilarityIndex(glove)

In [37]:
from re import sub
import nltk
from nltk.corpus import stopwords

from gensim.utils import simple_preprocess


stopwords = stopwords.words('english')

# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def preprocess(doc):
    # Tokenize, clean up input document string
#     doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
#     doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

# Preprocess the documents, including the query string

corpus = [preprocess(document) for document in dataset['Skills']]
print(corpus)
print()
query = preprocess(job_desc)
print(query)

[['matlab', 'electronics', 'arduino', 'creative', 'thinking', 'robotics', 'e', 'computer', 'engineering'], ['html', 'json', 'github', 'bootstrap', 'android', 'research', 'testing', 'web', 'design', 'rest', 'api', 'jquery', 'basic', 'python', 'java', 'e', 'js', 'django', 'ajax'], ['javascript', 'github', 'c', 'opengl', 'java', 'html', 'e', 'css'], ['processing', 'verilog', 'electronics', 'ml', 'data', 'processing', 'simulation', 'research', 'hdl', 'robotics', 'r'], ['simple', 'mysql', 'ide', 'nltk', 'javascript', 'github', 'android', 'bootstrap', 'git', 'python', 'basic', 'postgresql', 'pycharm', 'object', 'oriented', 'programming', 'html', 'css', 'django', 'numpy'], ['b', 'github', 'c', 'self', 'p', 'j', 'html', 'e', 'css', 'r'], ['nan'], ['python', 'nosql', 'analytics', 'e', 'k', 'sql', 'pandas', 'algorithms', 'p', 'data', 'structures', 'j', 'numpy', 'b', 'c', 'data', 'science', 'r'], ['leadership', 'research', 'content', 'creation', 'e', 'machine', 'learning', 'editing'], ['sql', 'ma

In [38]:
# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
print(dictionary)
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

Dictionary(499 unique tokens: ['arduino', 'computer', 'creative', 'e', 'electronics']...)


100%|████████████████████████████████████████████████████████████████████████████████| 499/499 [00:05<00:00, 90.01it/s]


In [39]:
# Finally, we calculate the soft cosine similarity between the query and each of the documents. Unlike the regular cosine similarity (which would return zero for vectors with no overlapping terms), the soft cosine similarity considers word similarity as well.


# Compute Soft Cosine Measure between the query and the documents.

query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)


doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes:
#     print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {new_df3['clean_overview'][idx]}')
    print((doc_similarity_scores[idx]),dataset["Skills"][idx])
    

# 1    0.688    tomatoes are actually fruit
# 0    0.000    cars drive on the road

0.8686218 Software development,Python,Jquery,Mongodb,Html,Javascript,Basic,Source,Ajax,Pl/sql,Sql,C++,Git,Css,Laravel,Vue.js,Bootstrap,Web development,C,Php
0.8487959 Matlab,Scrum,Python,Html,Analytics,Arduino,Javascript,Software testing,Testing,Basic,Java,E,Js,Netbeans,Mysql,Ide,C++,Wordpress,Pycharm,Css,Laravel,Github,Bootstrap,Agile,C,Php
0.8429587 Simulation,Adobe photoshop,Jquery,Html5,Arduino,Javascript,Testing,Basic,Java,E,Js,Atom,Netbeans,Mysql,Sql,Ide,Algorithms,Git,Data structures,Laravel,Github,Eclipse,Bootstrap,Android,C,Php
0.82347214 Scrum,Mern,Python,Mongodb,Html,Node.js,Editing,Arduino,Javascript,Express.js,Research,Postman,Java,React.js,Oop,Mysql,Wordpress,Leadership,Git,Css,Flutter,Collaboration,Machine learning,Bootstrap,Android,Opencv,C,Aws,Swift,Clean
0.82260776 Sql,Laravel,C++,Javascript,Github,Postman,Php,Self,Microsoft office,Java,Html,E,Js,Oop,S
0.81384206 Nosql,Python,Resnet,Node.js,Testbed,Arduino,Javascript,Basic,Java,Qt,E,React.js,Oop,Django,Small,Sql,C++,A